# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [5]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [6]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [7]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [8]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [9]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [10]:
np.linalg.matrix_rank(a1)

6

In [11]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


<ipython-input-5-8918e3eddeb9>:15: RuntimeWarning: divide by zero encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
<ipython-input-5-8918e3eddeb9>:17: RuntimeWarning: invalid value encountered in matmul
  u = lam @ u
<ipython-input-5-8918e3eddeb9>:20: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
<ipython-input-5-8918e3eddeb9>:15: RuntimeWarning: invalid value encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [12]:
#Linear algebra is importing for calculation of Minor
from numpy import linalg as la

def leading_minor(a):
  # construct Minor
  for i in range(N):
    b = np.zeros((i+1, i+1), dtype=float)
    for l in range(i+1):
      for k in range(i+1):
        b[l, k] = a[l, k]
    
    #print('Leading minors #',i,' is', la.det(b))
    # check Minor value
    if ( abs(la.det(b))<1e-15 ):
        print('Leading minors #',i+1,' is zero')
  print('\n')

print('Matrix a')
leading_minor(a)

print('Matrix a1')
leading_minor(a1)

Matrix a


Matrix a1
Leading minors # 2  is zero




### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [13]:
# Modify the diy_lu routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).
import numpy as np

def diy_lu_piv(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        piv = np.eye(N)
        i_max = j
        for l in range(j+1,N):
            c_max=u[j,j]
            if (u[l,j]<c_max):
                c_max= u[l,j]
                i_max=l
        if (i_max!=j):
            piv[j,j] = 0.
            piv[i_max,j] = 1.
            piv[i_max,i_max] = 0.
            piv[j,i_max] = 1.           
        u = piv @ u
    
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = (L @ piv.transpose()) @ lam
    return L, u

# Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices a and a1.
def reconstruct(L,u):
    a_orig= L @ u
    return a_orig

In [14]:
print("Original matrix\n",a, "\n")

L0, u0 = diy_lu_piv(a)

print("L:\n", L0, "\n")
print("u:\n", u0, "\n")

# print reconstruct the original matrix from a decompositon
print("Original matrix from a decompositon:\n",reconstruct(L0,u0), "\n")

print("Quick sanity check: L times U must equal the original matrix, up to floating-point errors.")
print(L0@u0 - a, "\n")

Original matrix
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

L:
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    0.5   1.    0.    0.    0.   ]
 [1.    0.727 0.706 2.35  3.387 1.   ]
 [1.    0.857 0.41  1.962 1.    0.   ]
 [1.    0.941 0.178 1.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]] 

u:
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  1.030e-02  2.020e-02  2.844e-02]
 [ 0.000e+00  2.220e-16  0.000e+00  0.000e+00  3.536e-04  8.163e-04]
 [ 0.000e+00 -7.521e-16  2.776e-17  0.000e+00  0.000e+00  2.890e-05]] 

Original matrix from a decompositon:
 [[3.    3.    3.    3.    3.    3.   ]
 [3.   